In [ ]:
from re import VERBOSE
# This line is very important on Colab. 
# Without it, Tensorflow 2.x will run and PINNs will not work.
%tensorflow_version 1.x

%xmode VERBOSE
# %pdb on

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys

# !pip install ipdb
# import ipdb
sys.path.insert(0, '../../Utilities/')

import math
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import time

print(tf.test.gpu_device_name())


using_gpu = tf.test.is_gpu_available(
  cuda_only=False, min_cuda_compute_capability=None
)

print(using_gpu)

if using_gpu:
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
np.random.seed(1234)
#tf.set_random_seed(1234)

class PhysicsInformedNN:
    # Initialize the class
    def __init__(self, X, u, layers, lb, ub):
        
        self.lb = lb
        self.ub = ub
        
        self.x = X[:,0:1]
        self.y = X[:,1:2]
        self.t = X[:,2:3]
        self.u = u
        
        self.layers = layers
        
        # Initialize NNs
        self.weights, self.biases = self.initialize_NN(layers)
        
        # tf placeholders and graph
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
       #self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     #log_device_placement=True))
        
        # Initialize parameters
        self.alpha_1 = tf.Variable([-1.8], dtype=tf.float32)
        
        self.x_tf = tf.placeholder(tf.float32, shape=[None, self.x.shape[1]])
        self.y_tf = tf.placeholder(tf.float32, shape=[None, self.y.shape[1]])
        self.t_tf = tf.placeholder(tf.float32, shape=[None, self.t.shape[1]])
        self.u_tf = tf.placeholder(tf.float32, shape=[None, self.u.shape[1]])
                
        self.u_pred = self.net_u(self.x_tf,self.y_tf, self.t_tf)
        self.f_pred = self.net_f(self.x_tf,self.y_tf, self.t_tf)
        
        self.loss = tf.reduce_mean(tf.square(self.u_tf - self.u_pred)) + \
                    tf.reduce_mean(tf.square(self.f_pred))
        
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 50000,
                                                                           'maxfun': 50000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})
        
        self.loss_writer = open('/content/drive/MyDrive/Colab Notebooks/Colab/PINN_loss.txt', 'w')

        # self.optimizer_Adam = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.2, beta2=0.2) 
        self.optimizer_Adam = tf.train.AdamOptimizer() 
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
        
        init = tf.global_variables_initializer()
        self.sess.run(init)

    def initialize_NN(self, layers):        
        weights = []
        biases = []
        
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))

        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases, eps=1e-3):
        num_layers = len(weights) + 1

        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
          W = weights[l]
          b = biases[l]
          H_norm = tf.compat.v1.layers.batch_normalization(H)
          H = tf.nn.leaky_relu(tf.add(tf.matmul(H_norm, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.nn.relu(tf.add(tf.matmul(H, W), b))

        return Y
            
    def net_u(self,x,y,t):  
        u = self.neural_net(tf.concat([x,y,t],1), self.weights, self.biases)

        return u
    
    def net_f(self, x,y,t):
        alpha_1 = self.alpha_1        
        u = self.net_u(x,y,t)
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_y = tf.gradients(u, y)[0]
        u_xx = tf.gradients(u_x, x)[0]
        u_yy = tf.gradients(u_y, y)[0]
        f = u_t - alpha_1*(u_xx + u_yy)

        return f
    
    def callback(self, loss, alpha_1):
        self.loss_writer.write('Loss: %e, alpha1: %.5f' % (loss, alpha_1))
        print('Loss: %e, alpha1: %.5f' % (loss, alpha_1))
        
        
    def train(self, nIter):
        tf_dict = {self.x_tf: self.x, self.y_tf: self.y, self.t_tf: self.t, self.u_tf: self.u}

        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            
            # Print            
            if it % 10 == 0:
                elapsed = time.time() - start_time
                loss_value = self.sess.run(self.loss, tf_dict)
                alpha_1_value = self.sess.run(self.alpha_1)
                
                print('It: %d, Loss: %.3e, alpha_1: %.3f, Time: %.2f' % 
                      (it, loss_value, alpha_1_value, elapsed))
                
                self.loss_writer.write('It: %d, Loss: %.3e, alpha_1: %.3f, Time: %.2f \n' % 
                      (it, loss_value, alpha_1_value, elapsed))
                
                start_time = time.time()
        
        
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss, self.alpha_1],
                                loss_callback = self.callback)
        
        self.loss_writer.close()
        
    def predict(self, X_star):
        
        tf_dict = {self.x_tf: X_star[:,0:1],self.y_tf: X_star[:,1:2], self.t_tf: X_star[:,2:3]}
        
        u_star = self.sess.run(self.u_pred, tf_dict)
        f_star = self.sess.run(self.f_pred, tf_dict)
        
        return u_star, f_star

In [ ]:
#if __name__ == "__main__": 
     
    #nu = 0.01/np.pi

N_u = 1000000# 853000
layers = [3, 20, 20, 20, 20, 20, 20, 20, 20, 1] # added last 4 hidden layers
    
input = np.load('/content/drive/MyDrive/Colab Notebooks/Colab/input.npy') # X1, X2 and T as three columns
output = np.load('/content/drive/MyDrive/Colab Notebooks/Colab/output.npy') # This is the solution of 2-D heat equation
output = output.reshape(output.shape[0],1) # changes shape from (n, ) to (n, 1)
    
X_star = input #input is coming from the 2Dheat.py file
u_star = output.flatten()[:,None] #output is coming from the 2Dheat.py file 

# Domain bounds
lb = input.min(0)
ub = input.max(0)

print(lb)
print(ub)

In [ ]:
start_point = 1600
stop_point = 1844799
sa = np.linspace(start_point, stop_point, stop_point-start_point)
# t=0 values are preseved in the original dataset as they are intial conditions
idx_sample = np.random.choice(sa, N_u, replace=False)

# New train Data for 2-D Heat Equation
X_u_train = np.vstack((input[0:1600, :], input[idx_sample.astype(int), :]))
u_train = np.concatenate([output[0:1600], output[idx_sample.astype(int)]])

print(X_u_train.min(0))
print(X_u_train.max(0))

In [ ]:
######################################################################
######################## Noiseless Data ##############################
######################################################################
noise = 0.0            
         
# idx = np.random.choice(input.shape[0], N_u, replace=False)
# X_u_train = input[idx,:]
# u_train = u_star[idx,:]

# np.where(input[:,2] >= 0.09)

start_point = 1600
stop_point = 1844799
sa = np.linspace(start_point, stop_point, stop_point-start_point)
# t=0 values are preseved in the original dataset as they are intial conditions
idx_sample = np.random.choice(sa, N_u, replace=False)

# New train Data for 2-D Heat Equation
X_u_train = np.vstack((input[0:1600, :], input[idx_sample.astype(int), :]))
u_train = np.concatenate([output[0:1600], output[idx_sample.astype(int)]])

model = PhysicsInformedNN(X_u_train, u_train, layers, lb, ub)
model.train(10000)#You can run for small iterations first, to see whether it is running properly

u_pred, f_pred = model.predict(X_star)
        
error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
    
alpha_1_value = model.sess.run(model.alpha_1)
error_alpha_1 = np.abs(alpha_1_value - 2)*100

print('Error u: %e' % (error_u))    
print('Error l1: %.5f%%' % (error_alpha_1)) 

df = open('output.txt', 'w')
df.write('Error u: %e' % (error_u))  
df.write('\n')
df.close()

np.save('/content/drive/MyDrive/Colab Notebooks/Colab//u_train-21March.npy', u_train)

u_pred_3D = np.reshape(u_pred, (1600, 40, 40))
np.save('/content/drive/MyDrive/Colab Notebooks/Colab/u_pred_3D-21March.npy', u_pred_3D)

In [ ]:
# %debug 

In [ ]:
######################################################################
############################ Animation ###############################
###################################################################### 

# Animation imports
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import matplotlib.pyplot as plt
%matplotlib inline

# plt.rc('text', usetex=True)
# matplotlib.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}']
# !apt install texlive-fonts-recommended texlive-fonts-extra cm-super dvipng
!pip install ffmpeg-python 


u_pred_3D = np.reshape(u_pred, (1600, 40, 40))
# u_pred_3D = np.load('/content/drive/MyDrive/Colab Notebooks/Colab/u_pred_3D-18March2022.npy')

plate_length = 1  # 10
delta_x = 0.025

alpha = 2

num_x_inter = int((plate_length)/delta_x)
max_iter_time = num_x_inter * num_x_inter

delta_t = (delta_x ** 2)/(4 * alpha)

tick_intervals = np.linspace(0, 40, 11, retstep=True)[0]
tick_labels = np.array(
['0.0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1.0'])


def plotheatmap(u_k, k):
  # Clear the current plot figure
  plt.clf()

  plt.title(f"Temperature at t = {k*delta_t:.3f} unit time")

  plt.xlabel("x")
  plt.xticks(tick_intervals, tick_labels)

  plt.ylabel("y")
  plt.yticks(tick_intervals, tick_labels)

        # This is to plot u_k (u at time-step k)
  plt.pcolormesh(u_k, cmap=plt.cm.jet, vmin=0, vmax=100)
  plt.colorbar()

  return plt


def animate(k):
  plotheatmap(u_pred_3D[k], k)


anim = animation.FuncAnimation(
    plt.figure(), animate, interval=1, frames=max_iter_time, repeat=False)
anim.save("/content/drive/MyDrive/Colab Notebooks/Colab/Images/predicted_heat_equation_solution-18March2022-2.gif", codec='gif', dpi=72, fps=100)